<a href="https://colab.research.google.com/github/CleanOOP/StudyAI/blob/Week4Homework/4weekBasicHomeWork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#이번 과제는 자연어 task 중 하나인 MNLI를 해결하는 모델을 HuggingFace로 학습하는 것입니다. MNLI를 요약하면 다음과 같습니다.

#- 입력: premise에 해당하는 문장과 hypothesis에 해당하는 문장 두 개가 입력으로 들어옵니다.
#- 출력: 분류 문제로, 두 문장이 들어왔을 때 다음 세 가지를 예측하시면 됩니다.
#    - Entailment: 두 문장에 논리적 모순이 없습니다.
#    - Neutral: 두 문장은 논리적으로 관련이 없습니다.
#    - Contradiction: 두 문장 사이에 논리적 모순이 존재합니다.

#이때, 다음 요구사항이 담긴 colab notebook을 만들어내시면 됩니다:

#-   `load_dataset("nyu-mll/glue", "mnli")` 로 dataset을 불러옵니다.
#    - 학습 때는 `train` split만 활용하셔야 합니다. 나머지 split은 사용불가입니다.
#    - Validation data가 필요한 경우, `train` split에서 가져오셔야 합니다.
#-   `trainer.train()`를 통해 학습된 log가 남아있어야 합니다.
#-   Dataset의 `validation_matched`에 대한 성능을 출력하고, 50%를 넘기셔야 합니다.

#이전 과제와 똑같이 validation data 유무, 모델 architecture, hyper-parameter 등은 위의 조건만 만족한다는 가정 하에서 마음대로 수정하셔도 됩니다.
## 제출자료

#위의 요구사항이 만족된 notebook을 public github repository에 업로드하여 링크를 제출합니다.
#반드시 출력 결과가 남아있어야 합니다

In [2]:
!pip install -qU datasets torchinfo openai

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 

DistilBert Finetuning by Huggingface Trainer
Task:
nyu-mll/glue의 mnli 데이터셋에 대한 distill bert의 분류 성능을 측정해야 합니다.
더 해볼 과제:
언어 모델 앙상블과 chain of thought prompt engineering을 통해 텍스트를 분류해볼 것입니다.

openai api 또한 간단히 사용할 예정입니다.

Distill bert로 분류한 것과 간단히 성능을 비교해보겠습니다.

In [4]:
import torch
# 모델 config입니다.
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model_name = "distilbert-base-uncased"
ds_name = "nyu-mll/glue"
ds_config = "mnli"

n_labels = 3

test_size1 = 0.1
test_size2 = 0.2222
random_state = 42
batch_size = 64
max_len = 400

results_dir = './results/week4_basic'
lr = 2e-5
n_epochs = 10

In [6]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from datasets import load_dataset
import pandas as pd #pandas : 데이터조작 및 분석용 sw lib

# Parquet 형태의 데이터셋입니다.
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=n_labels)
model = model.to(device)
raw_dataset = load_dataset(ds_name, ds_config)
label2idx = {'entailment':0,'neutral':1,'contradiction':2}

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
import torch
from torchinfo import summary
# 모델 정보 출력
vocab_size = len(tokenizer)
# vocab 안에서의 인덱스까지의 데이터
dummy_input_ids = torch.randint(0, vocab_size, (batch_size, max_len))
dummy_attention_mask = torch.ones(batch_size, max_len, dtype=torch.int64)

# 모델 요약 출력
print("Model Summary")
print("Vocab_size ", vocab_size)
print("Input Shape ", dummy_input_ids.shape)
print("Attention_Mask Shape ", dummy_attention_mask.shape)
print(summary(model, input_data={"input_ids": dummy_input_ids.to(device), "attention_mask": dummy_attention_mask.to(device)}))

Model Summary
Vocab_size  30522
Input Shape  torch.Size([64, 400])
Attention_Mask Shape  torch.Size([64, 400])
Layer (type:depth-idx)                                  Output Shape              Param #
DistilBertForSequenceClassification                     [64, 3]                   --
├─DistilBertModel: 1-1                                  [64, 400, 768]            --
│    └─Embeddings: 2-1                                  [64, 400, 768]            --
│    │    └─Embedding: 3-1                              [64, 400, 768]            23,440,896
│    │    └─Embedding: 3-2                              [1, 400, 768]             393,216
│    │    └─LayerNorm: 3-3                              [64, 400, 768]            1,536
│    │    └─Dropout: 3-4                                [64, 400, 768]            --
│    └─Transformer: 2-2                                 [64, 400, 768]            --
│    │    └─ModuleList: 3-5                             --                        42,527,232
├─Linear: 